In [18]:
import pandas as pd
import numpy as np
import os
import re
from xlrd import XLRDError
import datetime
from column_standard import variable_standardize
%matplotlib inline
import matplotlib.pyplot as plt

In [19]:
#first, we're going to run a STATA program that has already been written
#rather than duplicating that code, we'll run it inline here
#it takes the participant list and figures out how many participants have been added, deleted, and carried through this year for each ACO

#after running that STATA code, it outputs a file called added_deleted_byaco for each year
#we'll load 2016's because it will tell us who was added -and deleted in 2016 for the 2017 parlist
par_change = pd.read_stata(r"\\lmi.org\Data\Ser_Del\HlthMgmt\Civ\RstricOpen\CM608_Dellva\Working\608_Compliance_Data\ACO_Outlier_Analysis\Demographic_Data\added_deleted_byaco_{}.dta".format(datetime.datetime.now().year-1))
person_distros = pd.read_stata(r"\\lmi.org\Data\Ser_Del\HlthMgmt\Civ\RstricOpen\CM608_Dellva\Working\608_Compliance_Data\ACO_Outlier_Analysis\Demographic_Data\person_distros.dta")

risk_scores = pd.read_excel(r"\\lmi.org\Data\Ser_Del\HlthMgmt\Civ\RstricOpen\CM608_Dellva\Working\608_Compliance_Data\ACO_Outlier_Analysis\Demographic_Data\risk_scores.xlsx")

demographics = pd.merge(par_change,person_distros,left_on="ACO_ID",right_on="acoid",how="outer")
demographics.rename(columns={'acoid':'aco_id'},inplace=True)
del demographics["ACO_ID"]

demographics = pd.merge(demographics,risk_scores[risk_scores["year"]==2016],on="aco_id", how='left')

demographics

,Added,Deleted,CarriedThrough,TotalTINS_2017,aco_id,aco_name,aged_dual_person_years,aged_nondual_person_years,data_year_x,disabled_person_years,...,ageddual_risk_score,agednondual_risk_score,data_year_y,dis_risk_score,esrd_risk_score,year,ageddual_risk_score_mean,agednondual_risk_score_mean,dis_risk_score_mean,esrd_risk_score_mean
0,81.0,1.0,169.0,250.0,A1001,"Palm Beach Accountable Care Organization, LLC",4859.166667,59133.666667,2017.0,3887.416667,...,1.141094,1.231407,2016.0,1.243741,1.038517,2016.0,1.036896,1.077131,1.128267,1.029537
1,1.0,1.0,178.0,179.0,A1002,Chinese Community Accountable Care Organizatio...,7085.250000,1681.000000,2017.0,543.000000,...,0.823455,0.906926,2016.0,1.088410,0.948638,2016.0,1.036896,1.077131,1.128267,1.029537
2,10.0,3.0,13.0,23.0,A1006,"Hackensack Physician-Hospital Alliance ACO, LLC",2561.916667,28896.166667,2017.0,2526.333333,...,1.044222,1.107333,2016.0,1.148724,1.015224,2016.0,1.036896,1.077131,1.128267,1.029537
3,2.0,NaN,12.0,14.0,A1021,Concord Elliot ACO LLC,2408.000000,43622.416667,2017.0,6721.583333,...,1.103260,1.020278,2016.0,1.033902,1.029943,2016.0,1.036896,1.077131,1.128267,1.029537
4,2.0,2.0,8.0,10.0,A1023,"Arizona Connected Care, LLC",402.000000,5015.166667,2017.0,772.166667,...,0.741834,1.029450,2016.0,1.126055,0.959759,2016.0,1.036896,1.077131,1.128267,1.029537
5,40.0,3.0,48.0,88.0,A1026,"AHS ACO, LLC",2604.916667,49556.916667,2017.0,5169.166667,...,1.046273,1.068374,2016.0,1.057820,1.025700,2016.0,1.036896,1.077131,1.128267,1.029537
6,26.0,2.0,12.0,38.0,A1027,"Florida Physicians Trust, LLC",489.416667,7498.916667,2017.0,948.333333,...,1.318063,1.166239,2016.0,1.271408,1.100992,2016.0,1.036896,1.077131,1.128267,1.029537
7,7.0,5.0,30.0,37.0,A1032,"Premier ACO Physicians Network, LLC",1692.250000,5470.833333,2017.0,1097.083333,...,1.078479,1.156927,2016.0,1.269874,1.041891,2016.0,1.036896,1.077131,1.128267,1.029537
8,593.0,20.0,247.0,840.0,A1033,"Advocate Physician Partners Accountable Care, ...",7614.416667,117012.166670,2017.0,14750.916667,...,1.056270,1.057949,2016.0,1.082914,1.011758,2016.0,1.036896,1.077131,1.128267,1.029537
9,34.0,21.0,212.0,246.0,A1034,Catholic Medical Partners-Accountable Care IPA...,821.333333,9692.333333,2017.0,2314.166667,...,1.010281,1.176707,2016.0,1.042880,1.083206,2016.0,1.036896,1.077131,1.128267,1.029537


In [20]:
root = r'\\lmi.org\Data\Ser_Del\HlthMgmt\Civ\RstricOpen\CM608_Dellva\Working\608_Compliance_Data\RTI_Mainframe_Files\Quarterly Reports\\'

In [21]:
#bring in management agreement report which contains a bunch of descriptive data about the ACOs
downloads = r"\\lmi.org\Data\Ser_Del\HlthMgmt\Civ\RstricOpen\CM708_Dellva\Working\Compliance\Compliance_Data\Updated_HPMS_Downloads\\"
downloads_files = os.listdir(downloads)
for file in downloads_files:
    if "~$" not in file and "ManAgreement" in file:
        managree = pd.read_excel(downloads+file)
        managree = variable_standardize(managree)
        managree.aco_id = managree.aco_id.str.strip()
        managree = managree[['aco_id','termination_date','aco_legal_name','snf_3-day_rule_waiver_agreement_status','current_mssp_track']]
        managree.rename(columns={"snf_3-day_rule_waiver_agreement_status":"snf_waiver_status"}, inplace=True)
        managree.snf_waiver_status = managree.snf_waiver_status.str.strip()
        managree['track_split'] = np.where(managree['current_mssp_track'].str.slice(6,7) == "3", "Track 3", "Track 1/2")

managree_tracks = managree[['aco_id','track_split']]

demographics = pd.merge(managree,demographics,on="aco_id",how="left")

In [22]:
important_data = pd.read_excel(root + "\compiled_data.xlsx", sheetname = "important_data")

In [23]:
important_fields = ["aco_id",
"aco_name",
"data_year",
"mean_aged_dual_expenditures",
"mean_aged_non-dual_expenditures",
"mean_disabled_expenditures",
"mean_esrd_expenditures",
"mean_total_expenditures",
"quarter",
"hospital_discharges_total",
"skilled_nursing_facility_discharges",
"dme_expenditures",
"home_health_expenditures"
# ,"hospital_discharges_long-term_hospital",
# "hospital_discharges_short-term_hospital",
# "beneficiary_deaths",
# 'aged_dual_person_years',
# 'aged_non-dual_person_years',
# 'disabled_person_years',
# 'esrd_person_years',
# 'assigned_beneficiaries_total',
# 'part_b_expenditures_total'
]

In [24]:
important_field_renames = {"mean_aged_dual_expenditures_ratio":"Mean Aged Dual Expenditures",
"mean_aged_non-dual_expenditures_ratio":"Mean Aged Non-Dual Expenditures",
"mean_disabled_expenditures_ratio":"Mean Disabled Expenditures",
"mean_esrd_expenditures_ratio":"Mean ESRD Expenditures",
"mean_total_expenditures_ratio":"Mean Total Expenditures",
"hospital_discharges_total_ratio":"Total Hospital Discharges",
"skilled_nursing_facility_discharges_ratio":"SNF Discharges",
"dme_expenditures_ratio":"DME Expenditures",
"home_health_expenditures_ratio":"Home Health Expenditures"}

In [25]:
#Now, we're going to start doing the outlier analysis
#this is designed to ouput a list of outliers for that quarter
#as a note, you have to manually assign the most recent quarter 

#I HAVE NO IDEA WHY BUT DISCHARGES NEEDS TO BE FORCED TO FLOAT
#IT'S ALREADY FLOAT!
#DON'T ASK ME WHY WE NEED TO FORCE IT TO FLOAT SINCE ITS ALREADY FLOAT BUT I SPENT AN HOUR TRYIN TO DEBUG AND I'M OVER IT
important_data.hospital_discharges_total = important_data.hospital_discharges_total.astype(float)
important_data = pd.merge(managree_tracks,important_data, on="aco_id", how="outer")

#first, find the mean and standard deviation of all columns

important_means = important_data.groupby(['data_year','quarter','track_split'])
important_means = important_means.agg(['mean','std'])

#create a threshold for each variable based on those values
threshold = pd.DataFrame()
for var in important_means:
    threshold["{0}_threshold".format(var[0])] = important_means[var[0]]['mean']+important_means[var[0]]['std']*3
    threshold["{0}_mean".format(var[0])] = important_means[var[0]]['mean']
temp1 = pd.merge(important_data,threshold.reset_index(),on=['data_year','quarter','track_split'],suffixes=('', '_toDROP'))
for clm in temp1.columns:
    if "_toDROP" in clm:
        del temp1[clm]

outlier_list = {}
###FINDING OUTLIER LIST BY ACOID###

#group by acoid
temp1 = temp1.groupby("aco_id")

#iterate over all acoids
for name, grouped in temp1:
    print("Current Step: " + name)
    grouped.set_index(['data_year','quarter'], inplace=True)
    #for each acoid, iterate over the columns
    for var in [x for x in important_fields if x not in ["aco_id","aco_name",'data_year','quarter','track_split']]:
        if "{0}_threshold".format(var) in grouped.columns:
            print("Testing: {}".format(var))
            #dropping them if the value is under the threshold for 2017 Q1
            if 2015 in grouped.index:
                try:
                    if grouped.loc[2015,'Q4'][var] >= grouped.loc[2015,'Q4']["{0}_threshold".format(var)]:
                        print(var +" over threshold.")
                        grouped['{0}_ratio'.format(var)] = grouped.loc[:,:][var] / grouped.loc[:,:]["{0}_mean".format(var)]
                        grouped.drop("{0}_mean".format(var),inplace=True,axis=1)
                    else:
                        print("Dropping: "+var)
                        grouped.drop([var,"{0}_threshold".format(var),"{0}_mean".format(var)],inplace=True,axis=1)
                except KeyError:
                    continue
            else:
                continue
        else:
            grouped.drop(var,inplace=True,axis=1)
            print(var, "not in index")
            continue
        #if you drop all the columns, don't append it the following dict
        if len([x for x in grouped.columns if x not in ["aco_id","aco_name","data_year","quarter","track_split"]]) > 0:
            #store that list of columns to a dict with the acoid as the key
            outlier_list[name] = grouped
            
writer = pd.ExcelWriter(root + "\\outliers.xlsx")

#for each key in that dict, keep only the ratio of that category to the mean for that category in temp1
for aco in outlier_list.keys():
    if len([x for x in outlier_list[aco].columns if x not in ["aco_id","aco_name","data_year","quarter","track_split"]]) > 0:
    #store that dataframe in a dict with that acoid as a key
        outlier_list[aco].to_excel(writer,sheet_name=aco)
writer.save()

Current Step: A1001
Testing: mean_aged_dual_expenditures
Dropping: mean_aged_dual_expenditures
Testing: mean_aged_non-dual_expenditures
Dropping: mean_aged_non-dual_expenditures
Testing: mean_disabled_expenditures
Dropping: mean_disabled_expenditures
Testing: mean_esrd_expenditures
Dropping: mean_esrd_expenditures
Testing: mean_total_expenditures
Dropping: mean_total_expenditures
Testing: hospital_discharges_total
Dropping: hospital_discharges_total
Testing: skilled_nursing_facility_discharges
Dropping: skilled_nursing_facility_discharges
Testing: dme_expenditures
Dropping: dme_expenditures
Testing: home_health_expenditures
Dropping: home_health_expenditures
Current Step: A1002
Testing: mean_aged_dual_expenditures
Dropping: mean_aged_dual_expenditures


C:\Users\kredfield\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Testing: mean_aged_non-dual_expenditures
Dropping: mean_aged_non-dual_expenditures
Testing: mean_disabled_expenditures
Dropping: mean_disabled_expenditures
Testing: mean_esrd_expenditures
Dropping: mean_esrd_expenditures
Testing: mean_total_expenditures
Dropping: mean_total_expenditures
Testing: hospital_discharges_total
Dropping: hospital_discharges_total
Testing: skilled_nursing_facility_discharges
Dropping: skilled_nursing_facility_discharges
Testing: dme_expenditures
Dropping: dme_expenditures
Testing: home_health_expenditures
Dropping: home_health_expenditures
Current Step: A1003
Testing: mean_aged_dual_expenditures
Testing: mean_aged_non-dual_expenditures
Testing: mean_disabled_expenditures
Testing: mean_esrd_expenditures
Testing: mean_total_expenditures
Testing: hospital_discharges_total
Testing: skilled_nursing_facility_discharges
Testing: dme_expenditures
Testing: home_health_expenditures
Current Step: A1004
Testing: mean_aged_dual_expenditures
Testing: mean_aged_non-dual_expe

C:\Users\kredfield\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\kredfield\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Testing: hospital_discharges_total
Dropping: hospital_discharges_total
Testing: skilled_nursing_facility_discharges
Dropping: skilled_nursing_facility_discharges
Testing: dme_expenditures
Dropping: dme_expenditures
Testing: home_health_expenditures
Dropping: home_health_expenditures
Current Step: A1210
Testing: mean_aged_dual_expenditures
Testing: mean_aged_non-dual_expenditures
Testing: mean_disabled_expenditures
Testing: mean_esrd_expenditures
Testing: mean_total_expenditures
Testing: hospital_discharges_total
Testing: skilled_nursing_facility_discharges
Testing: dme_expenditures
Testing: home_health_expenditures
Current Step: A1211
Testing: mean_aged_dual_expenditures
Dropping: mean_aged_dual_expenditures
Testing: mean_aged_non-dual_expenditures
Dropping: mean_aged_non-dual_expenditures
Testing: mean_disabled_expenditures
Dropping: mean_disabled_expenditures
Testing: mean_esrd_expenditures
Dropping: mean_esrd_expenditures
Testing: mean_total_expenditures
Dropping: mean_total_expendi

In [32]:
writer = pd.ExcelWriter(root + "\\outliers_2015Q4.xlsx")

outliers = []
#for each key in that dict, keep only the ratio of that category to the mean for that category in temp1
for aco in outlier_list.keys():
    if len([x for x in outlier_list[aco].columns if x not in ["aco_id","aco_name","data_year","quarter","track_split",'inpatient_expenditures','inpatient_expenditures_threshold', 'inpatient_expenditures_mean']]) > 0:
    #store that dataframe in a dict with that acoid outlier_list[aco]as a key
        print(outlier_list[aco].columns)
        cols_to_use = [x for x in outlier_list[aco].columns if "_ratio" in x]
        temp = outlier_list[aco].reset_index()
        temp["data_year"] = temp["data_year"].round().astype(int)
        temp[aco] = temp["data_year"].map(str) + " " + temp["quarter"]
        cols_to_use.insert(0,aco)
        outliers.append(aco)
        quarters_to_use = ['2015 Q4','2015 Q3','2015 Q2','2015 Q1','2014 Q4']
        temp = temp[temp[aco].isin(quarters_to_use)]
        temp = temp[cols_to_use]
        temp.rename(columns=important_field_renames,inplace=True)
        temp.to_excel(writer,sheet_name=aco,index=False)
pd.DataFrame(outliers).to_excel(writer,sheet_name="All Outliers",index=False)
writer.save()

Index(['aco_id', 'track_split', 'aco_name', 'mean_disabled_expenditures',
       'inpatient_expenditures', 'mean_disabled_expenditures_threshold',
       'inpatient_expenditures_threshold', 'inpatient_expenditures_mean',
       'mean_disabled_expenditures_ratio'],
      dtype='object')
Index(['aco_id', 'track_split', 'aco_name', 'mean_aged_dual_expenditures',
       'hospital_discharges_total', 'skilled_nursing_facility_discharges',
       'inpatient_expenditures', 'mean_aged_dual_expenditures_threshold',
       'hospital_discharges_total_threshold',
       'skilled_nursing_facility_discharges_threshold',
       'inpatient_expenditures_threshold', 'inpatient_expenditures_mean',
       'mean_aged_dual_expenditures_ratio', 'hospital_discharges_total_ratio',
       'skilled_nursing_facility_discharges_ratio'],
      dtype='object')
Index(['aco_id', 'track_split', 'aco_name', 'mean_disabled_expenditures',
       'inpatient_expenditures', 'mean_disabled_expenditures_threshold',
       'inpa

In [27]:
writer = pd.to_excel(r"\\lmi.org\Data\Ser_Del\HlthMgmt\Civ\RstricOpen\CM608_Dellva\Working\Compliance_Data\ACO_Outlier_Analysis\2017_Q2\demographic_tables.xlsx", index=False)

AttributeError: module 'pandas' has no attribute 'to_excel'